In [ ]:
import geopandas as gpd

# 1. 정류소 데이터 로드 (GIS 파일 로드)
bus_gdf = gpd.read_file("C:/Users/User/big/부산광역시 버스 정류소 정보.zip")  # 정류소 데이터 GIS 파일 로드

# 2. 정류소 데이터의 좌표계를 EPSG:4326으로 설정 (WGS84)
bus_gdf = bus_gdf.set_crs(epsg=4326)

# 3. 행정구역 경계 데이터 로드
admin_boundary_dong = gpd.read_file("C:/Users/User/big/BND_ADM_DONG_PG.zip",encoding="cp949")  # 부산시 행정구역 경계 파일 로드

# 4. 좌표계 통일 (WGS84로 변환)
admin_boundary_dong = admin_boundary_dong.to_crs(epsg=4326)

# 5. 정류소와 행정동 간 공간 조인
# 'within' 대신 'intersects' 또는 'contains'를 사용할 수 있습니다.
bus_gdf_with_dong = gpd.sjoin(bus_gdf, admin_boundary_dong, how="left", predicate='within')
bus_gdf_with_dong = bus_gdf_with_dong.drop_duplicates(subset=['bstopnm', 'geometry'])

# 6. 결과 저장 (동별로 분류된 데이터 저장)
bus_gdf_with_dong[['bstopnm', 'geometry', 'ADM_NM']].to_csv("bus_stations_by_dong.csv", index=False, encoding='cp949')

print("동별로 분류된 데이터가 저장되었습니다.")

# 7. 동별 정류소 개수 집계
dong_station_counts = bus_gdf_with_dong.groupby('ADM_NM').size().reset_index(name='정류소 개수')

dong_station_counts
# 8. 결과 저장
dong_station_counts.to_csv("bus_station_counts_by_dong.csv", index=False, encoding='cp949')

print("동별 정류소 개수가 저장되었습니다.")